# Exercise 8

In [1]:
import numpy as np

np.random.seed(42)

In [2]:
def generate_embedded_reber():
    string = "B"
    if np.random.randint(2):
        string += "T" + generate_reber() + "T"
    else:
        string += "P" + generate_reber() + "P"
    return string + "E"

def generate_reber():
    string = "B"
    path = np.random.randint(2)
    
    if path:
        # upper path
        string += "T"
        state = 0
    else:
        # lower path
        string += "P"
        state = 2
    
    while state < 4:
        string, state = process_state(string, state)

    return string + "E"

def process_state(string, state):
    switch = np.random.randint(2)
    
    if state == 0:
        if switch:
            string += "S"
            state = 0
        else:
            string += "X"
            state = 1
    elif state == 1:
        if switch:
            string += "X"
            state = 2
        else:
            string += "S"
            state = 4
    elif state == 2:
        if switch:
            string += "T"
            state = 2
        else:
            string += "V"
            state = 3
    elif state == 3:
        if switch:
            string += "P"
            state = 1
        else:
            string += "V"
            state = 4
    else:
        raise ValueError("illegal input state")
    
    return string, state

In [3]:
def check_embedded_reber(string):
    if string[0] == "B":
        string = string[1:]
    else:
        return False
    
    path = string[0]
    string = string[1:]
    
    if path == "T":
        check, string = check_reber(string)
        if check and (string == "TE"):
            return True
    if path == "P":
        check, string = check_reber(string)
        if check and (string == "PE"):
            return True
    
    # in all other cases...
    return False

def check_reber(string):
    if string[0] == "B":
        string = string[1:]
    else:
        return False, " "
    
    path = string[0]
    string = string[1:]
    
    if path == "T":
        state = 0
    elif path == "P":
        state = 2
    else:
        return False, " "
    
    while state < 4:
        check, string, state = check_states(string, state)
        if not check or not string:
            return False, " "

    if string[0] == "E":
        return True, string[1:]
    
    # in all other cases...
    return False, " "

def check_states(string, state):
    char = string[0]
    string = string[1:]
    check = False

    if state == 0:
        if char == "S":
            state = 0
            check = True
        if char == "X":
            state = 1
            check = True
    elif state == 1:
        if char == "X":
            state = 2
            check = True
        if char == "S":
            state = 4
            check = True
    elif state == 2:
        if char == "T":
            state = 2
            check = True
        if char == "V":
            state = 3
            check = True
    elif state == 3:
        if char == "P":
            state = 1
            check = True
        if char == "V":
            state = 4
            check = True
    else:
        raise ValueError("illegal input state")
    
    return check, string, state

In [4]:
# generate some test string & check them
for string_idx in range(20):
    string = generate_embedded_reber()
    check = check_embedded_reber(string)
    print("{}: {}".format(string, check))

BPBTXSEPE: True
BPBTXSEPE: True
BPBTXSEPE: True
BPBPTVPXTVPSEPE: True
BTBTSSSSSSXSETE: True
BTBTSXXVVETE: True
BPBPVPXTTTVPXVPSEPE: True
BTBPTTVVETE: True
BPBPVVEPE: True
BPBPTTVPXTTVPSEPE: True
BTBTSXXVPSETE: True
BTBPVPSETE: True
BTBTSSSSSSSSSXSETE: True
BTBTSSSSSSXXVPXVPSETE: True
BTBTXXVPSETE: True
BPBTSXXTTVVEPE: True
BPBPVVEPE: True
BPBPVPSEPE: True
BTBTSXSETE: True
BPBPTVVEPE: True


In [5]:
alphabet = "BEPSTVX"
n_examples = 50000

examples = []
labels = np.empty(n_examples, dtype="uint8")


def corrupt_example(example):
    # make a negative example out of a positive one
    corrupt_chars = np.floor(len(example) / 2).astype("uint8")  # max 50% corrupt characters
    while check_embedded_reber(example):  # continue until we have a bad example
        s = list(example)
        for _ in range(corrupt_chars):
            corrupt_char = np.random.randint(len(example))
            replace_char = alphabet[np.random.randint(len(alphabet))]
            s[corrupt_char] = replace_char
        example = "".join(s)
    return example


for example_idx in range(n_examples):
    string = generate_embedded_reber()
    if np.random.randint(2):
        # add positive example
        examples.append(string)
        labels[example_idx] = 1
    else:
        examples.append(corrupt_example(string))
        labels[example_idx] = 0

In [6]:
# print some examples and labels

for idx in range(20):
    print("string: {}, label: {}".format(examples[idx], labels[idx]))

string: BPBPVPSEPE, label: 1
string: BPBTXSEPE, label: 1
string: BTBTXXVVETE, label: 1
string: BTBPEPPTXVETE, label: 0
string: BPBVXSTSEPE, label: 0
string: BPBPVPSEPE, label: 1
string: BPBPVSSTXVESEVE, label: 0
string: BTBPVVVPP, label: 0
string: EPBBXBEXE, label: 0
string: BEBTSXTVVEBV, label: 0
string: BVSTSTXTSTXVPSEBX, label: 0
string: BBSPTVSXBTE, label: 0
string: BPBTXSEPE, label: 1
string: BPBPTTVVEPE, label: 1
string: BTBTSXXTVPSETE, label: 1
string: BTBTXSETE, label: 1
string: BPPVVVESE, label: 0
string: VPBPTTTXSTVPXVVSPE, label: 0
string: BTBPTTVVETE, label: 1
string: BPBTSSSSXSEPE, label: 1


In [7]:
max([len(x) for x in examples])

43

(True, '', 3)